In [106]:
import numpy as np
import os
import sys

In [107]:
# Work-around for loading a module from a parent folder in Jupyter/Notebooks
parent_dir = os.path.abspath(os.path.join('..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from modules.utils import get_convolution_layer_expected_output_info

In [146]:
n_filters = 10
#padding = 'valid'  # No padding to the input
padding = 'same'    # Apply padding to the input

k=40
s=30

x_1d = np.random.rand(15, 120, 3)
k_1d = (k,)
s_1d = (s,)

x_2d = np.random.rand(15, 120, 120, 3)
k_2d = (k,k)
s_2d = (s,s)

x_3d = np.random.rand(15, 120, 120, 120, 3)
k_3d = (k,k,k)
s_3d = (s,s,s)

In [140]:
from tensorflow.keras.layers import Conv1D, Conv2D, Conv3D

def get_convolution_layer_output_info(input, kernel_shape, stride_shape, n_filters=10, padding='valid'):
    n_dims = len(input.shape[1:-1])
    if n_dims == 1:
        conv = Conv1D(
            filters=n_filters,
            kernel_size=kernel_shape,
            strides=stride_shape,
            padding=padding,
            name=f"conv_{n_dims}d_layer"
        )
    elif n_dims == 2:
        conv = Conv2D(
            filters=n_filters,
            kernel_size=kernel_shape,
            strides=stride_shape,
            padding=padding,
            name=f"conv_{n_dims}d_layer"
        )
    elif n_dims == 3:
        conv = Conv3D(
            filters=n_filters,
            kernel_size=kernel_shape,
            strides=stride_shape,
            padding=padding,
            name=f"conv_{n_dims}d_layer"
        )
    else:
        return (None,),(None,)
    y = conv(input)
    output_shape = y.shape[1:]

    #def get_padding_from_output(input, output, stride, kernel):
    #    p = (((output-1)*stride)+kernel-input)/2
    #    return int(p)
    #padding_shape = tuple([get_padding_from_output(input.shape[1:-1][i], ow, stride_shape[i], kernel_shape[i]) for i, ow in enumerate(output_shape[:-1])])

    #padding_shape = tuple([abs(int((((ow-1)*stride_shape[i])+kernel_shape[i]-input.shape[1:-1][i])/2)) for i, ow in enumerate(output_shape[:-1])])

    #return output_shape, padding_shape, y
    return output_shape, y

# Convolution 1D

In [147]:
x = x_1d
kernel_shape = k_1d
stride_shape = s_1d

name = 'Conv 1D'

print(f'Input shape ({name}): {x.shape[1:]}')

# Understaning the output shape of a Conv layer
expected_output_shape, expected_padding_shape = get_convolution_layer_expected_output_info(x, kernel_shape, stride_shape, n_filters, padding)
print(f'Expected output shape ({name}): {expected_output_shape}, padding: {expected_padding_shape}')

output_shape, y = get_convolution_layer_output_info(x, kernel_shape, stride_shape, n_filters, padding)
print(f'Output shape ({name}): {output_shape}')

Input shape (Conv 1D): (120, 3)
Expected output shape (Conv 1D): (4, 10), padding: (10,)
Output shape (Conv 1D): (4, 10)


# Convolution 2D

In [148]:
x = x_2d
kernel_shape = k_2d
stride_shape = s_2d

name = 'Conv 2D'

print(f'Input shape ({name}): {x.shape[1:]}')

# Understaning the output shape of a Conv layer
expected_output_shape, expected_padding_shape = get_convolution_layer_expected_output_info(x, kernel_shape, stride_shape, n_filters, padding)
print(f'Expected output shape ({name}): {expected_output_shape}, padding: {expected_padding_shape}')

output_shape, y = get_convolution_layer_output_info(x, kernel_shape, stride_shape, n_filters, padding)
print(f'Output shape ({name}): {output_shape}')

Input shape (Conv 2D): (120, 120, 3)
Expected output shape (Conv 2D): (4, 4, 10), padding: (10, 10)
Output shape (Conv 2D): (4, 4, 10)


# Convolution 3D

In [149]:
x = x_3d
kernel_shape = k_3d
stride_shape = s_3d

name = 'Conv 3D'

print(f'Input shape ({name}): {x.shape[1:]}')

# Understaning the output shape of a Conv layer
expected_output_shape, expected_padding_shape = get_convolution_layer_expected_output_info(x, kernel_shape, stride_shape, n_filters, padding)
print(f'Expected output shape ({name}): {expected_output_shape}, padding: {expected_padding_shape}')

output_shape, y = get_convolution_layer_output_info(x, kernel_shape, stride_shape, n_filters, padding)
print(f'Output shape ({name}): {output_shape}')

Input shape (Conv 3D): (120, 120, 120, 3)
Expected output shape (Conv 3D): (4, 4, 4, 10), padding: (10, 10, 10)
Output shape (Conv 3D): (4, 4, 4, 10)


# Tests

In [144]:
out_shape, pad_shape = get_convolution_layer_expected_output_info(np.random.rand(1, 28, 28, 32), (3,3), (2,2), n_filters=64, padding='same')
display(out_shape, pad_shape)

(14, 14, 64)

(1, 1)

In [145]:
out_shape, pad_shape = get_convolution_layer_expected_output_info(np.random.rand(1, 14, 14, 64), (3,3), (2,2), n_filters=64, padding='same')
display(out_shape, pad_shape)

(7, 7, 64)

(1, 1)